In [27]:
#Model : 3_OHTest7k

epochs = 2
path='bestmodel_3_OHTest7k.h5'
plot1 = 'accuracy_3_OHTest7k.jpg'
plot2 = 'loss_3_OHTest7k.jpg'
plot3 = 'f1_3_OHTest7k.jpg'
resultcsv = 'result_3_OHTest7k.csv'

In [28]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers

In [29]:
import keras as ks
import pandas as pd
import numpy as np
from PIL import Image
from keras.layers import Input
from keras.models import Model

In [30]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def dum(y_true,y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

In [31]:
df=pd.read_csv("affwildtrainNew2.csv")
df2=pd.read_csv("validation set AU.csv")

In [32]:
columns=['AU1_0','AU1_1','AU2_0','AU2_1','AU4_0','AU4_1','AU6_0','AU6_1','AU12_0','AU12_1','AU15_0','AU15_1','AU20_0','AU20_1','AU25_0','AU25_1']
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(dataframe=df[0:100],directory="AW2Train",x_col="ID",y_col=columns,batch_size=32,seed=42,shuffle=True,class_mode="other",target_size=(100,100))
valid_generator=test_datagen.flow_from_dataframe(dataframe=df[100:200],directory="AW2Train",x_col="ID",y_col=columns,batch_size=32,seed=42,shuffle=True,class_mode="other",target_size=(100,100))
test_generator=test_datagen.flow_from_dataframe(dataframe=df2[290:310],directory="validation set AU",x_col="ID",batch_size=1,seed=42,shuffle=False,class_mode=None,target_size=(100,100))
#train_generator[0]

Found 100 validated image filenames.
Found 100 validated image filenames.
Found 20 validated image filenames.


In [33]:
from keras.callbacks import Callback
from keras.backend import clear_session
from keras.models import Model, load_model
from keras.layers import Dense, Input, Flatten
from keras.applications import ResNet50



In [34]:
from keras.applications.resnet50 import ResNet50

In [35]:
#import sys
img = Input(shape = (100, 100, 3))
model = ResNet50(include_top=False, weights='imagenet', input_tensor=img, input_shape=(100,100,3), pooling='avg')
x = Dense(128, activation = 'relu')(model.output)


output1 = Dense(2, activation = 'softmax',name='AU1')(x)
output2 = Dense(2, activation = 'softmax',name='AU2')(x)
output3 = Dense(2, activation = 'softmax',name='AU4')(x)
output4 = Dense(2, activation = 'softmax',name='AU6')(x)
output5 = Dense(2, activation = 'softmax',name='AU12')(x)
output6 = Dense(2, activation = 'softmax',name='AU15')(x)
output7 = Dense(2, activation = 'softmax',name='AU20')(x)
output8 = Dense(2, activation = 'softmax',name='AU25')(x)

model = Model(img,[output1,output2,output3,output4,output5,output6,output7,output8])
model.compile(optimizers.adam(lr=0.001),
loss = ["categorical_crossentropy","categorical_crossentropy","categorical_crossentropy","categorical_crossentropy","categorical_crossentropy","categorical_crossentropy","categorical_crossentropy","categorical_crossentropy"],metrics = ["accuracy",f1_m,dum])
   

In [36]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 106, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 50, 50, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 50, 50, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [37]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,0:2],batch_y[:,2:4],batch_y[:,4:6],batch_y[:,6:8],batch_y[:,8:10],batch_y[:,10:12],batch_y[:,12:14],batch_y[:,14:16]])
        #yield (batch_x,[batch_y[:,0],batch_y[:,1],batch_y[:,2],batch_y[:,3].batch_y[:,4],batch_y[:,5],batch_y[:,6],batch_y[:,7]])

In [46]:
import tensorflow as tf
import keras as ks
class AvgF(tf.keras.callbacks.Callback):
    #def on_train_begin(self, logs={}):
    #    self.model.avgval_f1s = 0.0
    def on_epoch_end(self,epoch, logs=None):
        #val f1 
        logs['val_AU1_dum']=((logs['val_AU1_f1_m']+logs['val_AU2_f1_m']+logs['val_AU4_f1_m']+
                                     logs['val_AU6_f1_m']+logs['val_AU12_f1_m']+logs['val_AU15_f1_m']+
                                     logs['val_AU20_f1_m']+logs['val_AU25_f1_m'])/8.0)
        #train f1
        logs['val_AU2_dum']=((logs['AU1_f1_m']+logs['AU2_f1_m']+logs['AU4_f1_m']+
                                     logs['AU6_f1_m']+logs['AU12_f1_m']+logs['AU15_f1_m']+
                                     logs['AU20_f1_m']+logs['AU25_f1_m'])/8.0)
        #val acc
        logs['val_AU4_dum']=((logs['val_AU1_acc']+logs['val_AU2_acc']+logs['val_AU4_acc']+
                                     logs['val_AU6_acc']+logs['val_AU12_acc']+logs['val_AU15_acc']+
                                     logs['val_AU20_acc']+logs['val_AU25_acc'])/8.0)
        #train acc
        logs['val_AU6_dum']=((logs['AU1_acc']+logs['AU2_acc']+logs['AU4_acc']+
                                     logs['AU6_acc']+logs['AU12_acc']+logs['AU15_acc']+
                                     logs['AU20_acc']+logs['AU25_acc'])/8.0)
        
        print("Average train f1 score: ",logs['val_AU2_dum'])
        print("Average val f1 score: ",logs['val_AU1_dum'])
        print("Average train accuracy: ",logs['val_AU6_dum'])
        print("Average val accuracy: ",logs['val_AU4_dum'])
        
        return 

In [47]:
from keras.callbacks import ModelCheckpoint
avg=AvgF()
mc=ModelCheckpoint('one.h5', monitor='val_AU1_dum', mode='max', save_best_only=True)

In [48]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
history=model.fit_generator(generator=generator_wrapper(train_generator),steps_per_epoch=STEP_SIZE_TRAIN,validation_data=generator_wrapper(valid_generator),validation_steps=STEP_SIZE_VALID,epochs=2,verbose=1,
                            callbacks=[avg,mc])

Epoch 1/2
3/3 [==============================] - 12s 4s/step - loss: 0.8780 - AU1_loss: 0.8779 - AU2_loss: 4.2697e-05 - AU4_loss: 6.4731e-06 - AU6_loss: 1.1813e-05 - AU12_loss: 5.8355e-06 - AU15_loss: 1.4195e-05 - AU20_loss: 3.1771e-05 - AU25_loss: 5.0584e-05 - AU1_acc: 0.8879 - AU1_f1_m: 0.8879 - AU1_dum: 0.8879 - AU2_acc: 1.0000 - AU2_f1_m: 1.0000 - AU2_dum: 1.0000 - AU4_acc: 1.0000 - AU4_f1_m: 1.0000 - AU4_dum: 1.0000 - AU6_acc: 1.0000 - AU6_f1_m: 1.0000 - AU6_dum: 1.0000 - AU12_acc: 1.0000 - AU12_f1_m: 1.0000 - AU12_dum: 1.0000 - AU15_acc: 1.0000 - AU15_f1_m: 1.0000 - AU15_dum: 1.0000 - AU20_acc: 1.0000 - AU20_f1_m: 1.0000 - AU20_dum: 1.0000 - AU25_acc: 1.0000 - AU25_f1_m: 1.0000 - AU25_dum: 1.0000 - val_loss: 22.8582 - val_AU1_loss: 7.6508 - val_AU2_loss: 8.6632e-06 - val_AU4_loss: 2.9978e-07 - val_AU6_loss: 4.5660e-06 - val_AU12_loss: 15.2073 - val_AU15_loss: 5.8642e-06 - val_AU20_loss: 9.1815e-06 - val_AU25_loss: 3.5974e-06 - val_AU1_acc: 0.2059 - val_AU1_f1_m: 0.2059 - val_AU1_

In [49]:
print("Average val f1 score after epoch 1: ",history.history['val_AU1_dum'][0])
print("Average val f1 score after epoch 2: ",history.history['val_AU1_dum'][1])

('Average val f1 score after epoch 1: ', 0.7757352866670665)
('Average val f1 score after epoch 2: ', 0.8763020811602473)


In [50]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,8))
plt.plot(range(epochs),history.history['val_AU6_dum'],label='training accuracy', color = "blue")
plt.plot(range(epochs),history.history['val_AU4_dum'],label='validation accuracy', color = "red")
plt.legend(loc=0)
plt.xlabel('epochs')
plt.xlim([0,epochs])
plt.ylabel('accuracies on dataset')
plt.grid(True)
plt.title("Training and validation accuracy")
plt.show()
fig.savefig(plot1)
plt.close(fig)
fig = plt.figure(figsize=(12,8))
plt.plot(range(epochs),history.history['loss'],label='training loss')
plt.plot(range(epochs),history.history['val_loss'],label='validation loss')
plt.legend(loc=0)
plt.xlabel('epochs')
plt.xlim([0,epochs])
plt.ylabel('losses on dataset')
plt.grid(True)
plt.title("Training and validation loss")
plt.show()
fig.savefig(plot2)
plt.close(fig)
fig = plt.figure(figsize=(12,8))
plt.plot(range(epochs),history.history['val_AU2_dum'],label='training f1 score')
plt.plot(range(epochs),history.history['val_AU1_dum'],label='validation f1 score')
plt.legend(loc=0)
plt.xlabel('epochs')
plt.xlim([0,epochs])
plt.ylabel('f1 scores on dataset')
plt.grid(True)
plt.title("Training and validation f1 score")
plt.show()
fig.savefig(plot3)
plt.close(fig)

<Figure size 1200x800 with 1 Axes>

<Figure size 1200x800 with 1 Axes>

<Figure size 1200x800 with 1 Axes>

In [15]:
from keras.models import load_model
model1 = load_model(path,custom_objects={'f1_m':f1_m})

In [16]:
#test=train
df3=df2[60000:]
y_test= df3.drop(['ID'], axis = 1)
y_test = y_test.to_numpy()
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(dataframe=df3,directory="validation set AU",x_col="ID",batch_size=1,seed=42,shuffle=False,class_mode=None,target_size=(100,100))
STEP_SIZE_TEST=test_generator.n

y_pred=model1.predict_generator(test_generator,steps=STEP_SIZE_TEST,verbose=1)

Found 7106 validated image filenames.
7106/7106 [==============================] - 407s 57ms/step


In [17]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [ ]:
num=test_generator.n
for i in range(num):
    for j in range(2):
        if(y_pred[0][i][j]>0.5):
            y_pred[0][i][j]=1
        else:
            y_pred[0][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[1][i][j]>0.5):
            y_pred[1][i][j]=1
        else:
            y_pred[1][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[2][i][j]>0.5):
            y_pred[2][i][j]=1
        else:
            y_pred[2][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[3][i][j]>0.5):
            y_pred[3][i][j]=1
        else:
            y_pred[3][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[4][i][j]>0.5):
            y_pred[4][i][j]=1
        else:
            y_pred[4][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[5][i][j]>0.5):
            y_pred[5][i][j]=1
        else:
            y_pred[5][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[6][i][j]>0.5):
            y_pred[6][i][j]=1
        else:
            y_pred[6][i][j]=0
for i in range(num):
    for j in range(2):
        if(y_pred[7][i][j]>0.5):
            y_pred[7][i][j]=1
        else:
            y_pred[7][i][j]=0


In [ ]:
num=test_generator.n
rows, cols = (num, 8) 
arr2 = [[int(0)]*cols]*rows
arr2=np.array(arr2)
for i in range(num):
    if(y_pred[0][i][0].astype(int)==1):
        arr2[i][0]=0
    else:
        arr2[i][0]=1
for i in range(num):
    if(y_pred[1][i][0].astype(int)==1):
        arr2[i][1]=0
    else:
        arr2[i][1]=1
for i in range(num):
    if(y_pred[2][i][0].astype(int)==1):
        arr2[i][2]=0
    else:
        arr2[i][2]=1
for i in range(num):
    if(y_pred[3][i][0].astype(int)==1):
        arr2[i][3]=0
    else:
        arr2[i][3]=1
for i in range(num):
    if(y_pred[4][i][0].astype(int)==1):
        arr2[i][4]=0
    else:
        arr2[i][4]=1
for i in range(num):
    if(y_pred[5][i][0].astype(int)==1):
        arr2[i][5]=0
    else:
        arr2[i][5]=1
for i in range(num):
    if(y_pred[6][i][0].astype(int)==1):
        arr2[i][6]=0
    else:
        arr2[i][6]=1
for i in range(num):
    if(y_pred[7][i][0].astype(int)==1):
        arr2[i][7]=0
    else:
        arr2[i][7]=1
y_pred=arr2

In [19]:
num=test_generator.n
print("Metrics for AU1\n")
y_test_AU1 = [0.0]*num
y_pred_AU1=[0.0]*num
for i in range(num):
    y_test_AU1[i]=y_test[i][0]
    y_pred_AU1[i]=y_pred[i][0]
print("\nTrue values\n")
print(y_test_AU1)
print("\nPredicted values\n")
print(y_pred_AU1)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU1, y_pred_AU1))
print(metrics.classification_report(y_test_AU1, y_pred_AU1))

Metrics for AU1


True values

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [48]:
print("Metrics for AU2\n")
y_test_AU2 = [0.0]*num
y_pred_AU2=[0.0]*num
for i in range(num):
    y_test_AU2[i]=y_test[i][1]
    y_pred_AU2[i]=y_pred[i][1]
print("\nTrue values\n")
print(y_test_AU2)
print("\nPredicted values\n")
print(y_pred_AU2)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU2, y_pred_AU2))
print(metrics.classification_report(y_test_AU2, y_pred_AU2))

Metrics for AU2


True values

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [49]:
print("Metrics for AU4\n")
y_test_AU4 = [0.0]*num
y_pred_AU4=[0.0]*num
for i in range(num):
    y_test_AU4[i]=y_test[i][2]
    y_pred_AU4[i]=y_pred[i][2]
print("\nTrue values\n")
print(y_test_AU4)
print("\nPredicted values\n")
print(y_pred_AU4)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU4, y_pred_AU4))
print(metrics.classification_report(y_test_AU4, y_pred_AU4))

Metrics for AU4


True values

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [50]:
print("Metrics for AU6\n")
y_test_AU6 = [0.0]*num
y_pred_AU6=[0.0]*num
for i in range(num):
    y_test_AU6[i]=y_test[i][3]
    y_pred_AU6[i]=y_pred[i][3]
print("\nTrue values\n")
print(y_test_AU6)
print("\nPredicted values\n")
print(y_pred_AU6)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU6, y_pred_AU6))
print(metrics.classification_report(y_test_AU6, y_pred_AU6))

Metrics for AU6


True values

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [51]:
print("Metrics for AU12\n")
y_test_AU12 = [0.0]*num
y_pred_AU12=[0.0]*num
for i in range(num):
    y_test_AU12[i]=y_test[i][4]
    y_pred_AU12[i]=y_pred[i][4]
print("\nTrue values\n")
print(y_test_AU12)
print("\nPredicted values\n")
print(y_pred_AU12)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU12, y_pred_AU12))
print(metrics.classification_report(y_test_AU12, y_pred_AU12))

Metrics for AU12


True values

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [52]:
print("Metrics for AU15\n")
y_test_AU15 = [0.0]*num
y_pred_AU15=[0.0]*num
for i in range(num):
    y_test_AU15[i]=y_test[i][5]
    y_pred_AU15[i]=y_pred[i][5]
print("\nTrue values\n")
print(y_test_AU15)
print("\nPredicted values\n")
print(y_pred_AU15)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU15, y_pred_AU15))
print(metrics.classification_report(y_test_AU15, y_pred_AU15))

Metrics for AU15


True values

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [53]:
print("Metrics for AU20\n")
y_test_AU20 = [0.0]*num
y_pred_AU20=[0.0]*num
for i in range(num):
    y_test_AU20[i]=y_test[i][6]
    y_pred_AU20[i]=y_pred[i][6]
print("\nTrue values\n")
print(y_test_AU20)
print("\nPredicted values\n")
print(y_pred_AU20)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU20, y_pred_AU20))
print(metrics.classification_report(y_test_AU20, y_pred_AU20))

Metrics for AU20


True values

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [54]:
print("Metrics for AU25\n")
y_test_AU25 = [0.0]*num
y_pred_AU25=[0.0]*num
for i in range(num):
    y_test_AU25[i]=y_test[i][7]
    y_pred_AU25[i]=y_pred[i][7]
print("\nTrue values\n")
print(y_test_AU25)
print("\nPredicted values\n")
print(y_pred_AU25)
from sklearn import metrics
print("\nconfusion matrix\n")
print(metrics.confusion_matrix(y_test_AU25, y_pred_AU25))
print(metrics.classification_report(y_test_AU25, y_pred_AU25))

Metrics for AU25


True values

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [25]:
columns=["AU1", "AU2", "AU4", "AU6", "AU12","AU15","AU20","AU25"]
#columns should be the same order of y_col
results=pd.DataFrame(y_pred, columns=columns)
results["Filenames"]=test_generator.filenames
ordered_cols=["Filenames"]+columns
results=results[ordered_cols]#To get the same column order
results.to_csv(resultcsv,index=False)

NameError: name 'y_pred' is not defined